### Necessary imports

In [ ]:
# !pip install Levenshtein
import Levenshtein
import json
import nltk
import pandas as pd
import re 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 258 kB 14.4 MB/s 
     |████████████████████████████████| 1.8 MB 68.0 MB/s 
     |████████████████████████████████| 103 kB 54.4 MB/s 


In [ ]:
!gdown --id 1KOrKTVqhDZvDgheBdG5ijn8RFqaTGMnp #training
!gdown --id 1_TX7l9miXEe_r30YioSwPLndk0rgiWkt #sample dev predictions
!gdown --id 1jo4l_DfhSsUXH3KSvctQ3DwLywTAxczZ #dev

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1KOrKTVqhDZvDgheBdG5ijn8RFqaTGMnp
To: /content/WIESP2022-NER-TRAINING.jsonl
100% 9.95M/9.95M [00:00<00:00, 186MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1_TX7l9miXEe_r30YioSwPLndk0rgiWkt
To: /content/WIESP2022-NER-DEV-sample-predictions.jsonl
100% 177k/177k [00:00<00:00, 39.7MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  cat

In [ ]:
with open("WIESP2022-NER-TRAINING.jsonl", 'r') as f:
  wiesp_train_json = [json.loads(l) for l in list(f)]

In [ ]:
df_to_preproc = pd.DataFrame(wiesp_train_json)
df_to_preproc.head()

,bibcode,label_studio_id,ner_ids,ner_tags,section,tokens,unique_id
0,2019MNRAS.486.5558S,487,"[62, 62, 62, 62, 62, 62, 62, 15, 62, 62, 62, 6...","[O, O, O, O, O, O, O, B-Instrument, O, O, O, O...",fulltext,"[Whilst, a, reasonable, harmonic, fit, to, the...",fulltext_487_2019MNRAS.486.5558S
1,2018MNRAS.478.5533F,1129,"[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 6...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",fulltext,"[comparison, once, the, angular, positions, of...",fulltext_1129_2018MNRAS.478.5533F
2,2018MNRAS.480.3062L,1086,"[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 6...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",acknowledgments,"[ACKNOWLEDGEMENTS, The, authors, thank, an, an...",acknowledgments_1086_2018MNRAS.480.3062L
3,2016MNRAS.457.1786M,1135,"[22, 62, 62, 62, 62, 21, 13, 44, 44, 21, 62, 1...","[B-Person, O, O, O, O, B-Organization, B-Grant...",acknowledgments,"[BDM, gratefully, acknowledges, support, from,...",acknowledgments_1135_2016MNRAS.457.1786M
4,2019MNRAS.482L...9B,559,"[62, 62, 62, 22, 53, 22, 53, 22, 53, 22, 53, 2...","[O, O, O, B-Person, I-Person, B-Person, I-Pers...",acknowledgments,"[ACKNOWLEDGEMENTS, We, thank, Dougal, Mackey,,...",acknowledgments_559_2019MNRAS.482L...9B


In [ ]:
df_to_preproc.bibcode.nunique()

1723

In [ ]:
temp_df = df_to_preproc.loc[:5]

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preproc(token_list):

  better_token_list = []
  stopwords_eng = stopwords.words('english')
  stemmer = PorterStemmer()


  for word in token_list:
    
    word = re.sub(r'[&]', "and", re.sub(r'[!@#`~_{}.,;()\[\]?]',"",word)).lower()
    # if word not in stopwords_eng:
    #   better_token_list.append(stemmer.stem(word))
    # else:
    #   better_token_list.append("<stopword>")
    better_token_list.append(stemmer.stem(word))
  
  return(better_token_list)

In [ ]:
preproc(["Token_list", "12gdh@67##dd_##[]0.a", "&", 'happening'])

['tokenlist', '12gdh67dd0a', 'and', 'happen']

In [ ]:
df_to_preproc['tokens'] = df_to_preproc['tokens'].apply(preproc)

In [ ]:
def sugg_preproc(token_list):

  better_token_list = []
  stopwords_eng = stopwords.words('english')
  stemmer = PorterStemmer()


  for word in token_list:
    
    word = re.sub(r'[&]', "and", re.sub(r'[!@#`~_{}.,;()\[\]?]',"",word)).lower()
    # if word not in stopwords_eng:
    #   better_token_list.append(stemmer.stem(word))
    # else:
    #   better_token_list.append("<stopword>")
    better_token_list.append(word)
  
  return(better_token_list)

In [ ]:
for_sugg = pd.DataFrame(wiesp_train_json)['tokens'].apply(sugg_preproc)

In [ ]:
df_to_preproc.head()

,bibcode,label_studio_id,ner_ids,ner_tags,section,tokens,unique_id
0,2019MNRAS.486.5558S,487,"[62, 62, 62, 62, 62, 62, 62, 15, 62, 62, 62, 6...","[O, O, O, O, O, O, O, B-Instrument, O, O, O, O...",fulltext,"[whilst, a, reason, harmon, fit, to, the, espa...",fulltext_487_2019MNRAS.486.5558S
1,2018MNRAS.478.5533F,1129,"[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 6...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",fulltext,"[comparison, onc, the, angular, posit, of, the...",fulltext_1129_2018MNRAS.478.5533F
2,2018MNRAS.480.3062L,1086,"[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 6...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",acknowledgments,"[acknowledg, the, author, thank, an, anonym, r...",acknowledgments_1086_2018MNRAS.480.3062L
3,2016MNRAS.457.1786M,1135,"[22, 62, 62, 62, 62, 21, 13, 44, 44, 21, 62, 1...","[B-Person, O, O, O, O, B-Organization, B-Grant...",acknowledgments,"[bdm, grate, acknowledg, support, from, nasa, ...",acknowledgments_1135_2016MNRAS.457.1786M
4,2019MNRAS.482L...9B,559,"[62, 62, 62, 22, 53, 22, 53, 22, 53, 22, 53, 2...","[O, O, O, B-Person, I-Person, B-Person, I-Pers...",acknowledgments,"[acknowledg, we, thank, dougal, mackey, adrian...",acknowledgments_559_2019MNRAS.482L...9B


In [ ]:
from collections import Counter
def vocab_builder(col=df_to_preproc.tokens):
  word_list=[]
  for lst in col:
    word_list+=lst
  
  return Counter(word_list)

# vocab_builder()  

In [ ]:
test = vocab_builder(for_sugg)
# test

In [ ]:
vocab_count_dict = vocab_builder()
total_word_counts = sum(vocab_count_dict.values())

In [ ]:
vocab_proba = {k:v/total_word_counts for k,v in vocab_count_dict.items()}

In [ ]:
# vocab_proba

In [ ]:
word = "nuetrul"


char_list = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                 '1','2','3','4','5','6','7','8','9','0','-','%','$','^',':','/','*']

p_splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
r_spilts = [(word[:i-1],word[i-1:i], word[i:]) for i in range(1,len(word)+1,1)]
sw_splits = [(word[:i-2],word[i-2:i], word[i:]) for i in range(2,len(word)+1,1)]

In [ ]:
d_combos = [L + R[1:] for L, R in p_splits if R]
i_combos = [L+alpha+R[1:] for L, R in p_splits for alpha in char_list]
r_combos = [L+alpha+R for L, _, R in sw_splits for alpha in char_list]
sw_combos = [L+M[::-1]+R for L, M, R in sw_splits]


In [ ]:
def one_edit(word:str, top5=True):
  char_list = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                 '1','2','3','4','5','6','7','8','9','0','-','%','$','^',':','/','*']

  
  p_splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
  r_spilts = [(word[:i-1],word[i-1:i], word[i:]) for i in range(1,len(word)+1,1)]
  sw_splits = [(word[:i-2],word[i-2:i], word[i:]) for i in range(2,len(word)+1,1)]

  d_combos = [L + R[1:] for L, R in p_splits if R]
  i_combos = [L+alpha+R[1:] for L, R in p_splits for alpha in char_list]
  r_combos = [L+alpha+R for L, _, R in sw_splits for alpha in char_list]
  sw_combos = [L+M[::-1]+R for L, M, R in sw_splits]

  final_combos = [word for word in set(d_combos+i_combos+r_combos+sw_combos) if word in test.keys()]
  if top5:
    return(final_combos[:5])
  else:
    return(final_combos)
# i_combos

In [ ]:
one_edit('bart')

['bae', 'xrt', 'bao', 'bpt', 'part']

In [ ]:
def two_edit(word:str, top5=True):
  char_list = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                 '1','2','3','4','5','6','7','8','9','0','-','%','$','^',':','/','*']

  
  p_splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
  r_spilts = [(word[:i-1],word[i-1:i], word[i:]) for i in range(1,len(word)+1,1)]
  sw_splits = [(word[:i-2],word[i-2:i], word[i:]) for i in range(2,len(word)+1,1)]

  d_combos = [L + R[1:] for L, R in p_splits if R]
  d2_combos = []
  for word_ in d_combos:
    d2_combos += one_edit(word_, top5=False)

  i_combos = [L+alpha+R[1:] for L, R in p_splits for alpha in char_list]
  i2_combos = []
  for word_ in i_combos:
    i2_combos += one_edit(word_, top5=False)

  r_combos = [L+alpha+R for L, _, R in sw_splits for alpha in char_list]
  r2_combos = []
  for word_ in r_combos:
    r2_combos += one_edit(word_, top5=False)
  

  sw_combos = [L+M[::-1]+R for L, M, R in sw_splits]
  sw2_combos = []
  for word_ in sw_combos:
    sw2_combos += one_edit(word_, top5=False)

  final_combos = list(set([word for word in set(d2_combos+i2_combos+r2_combos+sw2_combos) if word in test.keys()]))

  dist_dict = {}
  for word_ in final_combos:
    dist_dict[word_] = Levenshtein.distance(word, word_)
  res = [k for k, _ in sorted(dist_dict.items(), key=lambda x: x[1]) if _ <3]
  
  if top5:
    return(res[:5])
  else:
    return(res)

In [ ]:
two_edit('bart') , one_edit('bart')

(['bar', 'barth', 'part', 'bat', 'art'], ['bae', 'xrt', 'bao', 'bpt', 'part'])

In [ ]:
def all_edits(word, top5=True):
  
  edit1_list=[]
  edit2_list=[]
  
  edit1_list=one_edit(word)
  dist_dict = {}
  for word_ in edit1_list:
    dist_dict[word_] = Levenshtein.distance(word, word_)
  edit1_list = [k for k, _ in sorted(dist_dict.items(), key=lambda x: x[1]) if _ <3] 

  edit2_list=two_edit(word)

  res_suggest = list(set(edit1_list+edit2_list))

  if top5:
    return(res_suggest[:5])
  else:
    return(res_suggest)

In [ ]:
all_edits("neutrl")

['neutral', 'neutron', 'neural']

In [ ]:
#sentence needs to be stemmed

def check_input(sentence:str):
  list_for_correction_check = re.findall('\w+', sentence)
  for word in list_for_correction_check:
    if word not in vocab_count_dict.keys():
      s = all_edits(word)
      print("(Stemmed Alternatives) Do you mean any of",s," instead of "+word+" ?")
      val = input("which position was the correct word in?[1-n]")
      if val:
          list_for_correction_check[list_for_correction_check.index(word)]=s[int(val)-1]

  print("The corrected query thus is, ",list_for_correction_check)
  return(list_for_correction_check)

In [ ]:
check_input("we, thank, dogual, mackeyy, adrian")

(Stemmed Alternatives) Do you mean any of ['dagal', 'dougal', 'dual']  instead of dogual ?
which position was the correct word in?[1-n]2
(Stemmed Alternatives) Do you mean any of ['mackey']  instead of mackeyy ?
which position was the correct word in?[1-n]1
The corrected query thus is,  ['we', 'thank', 'dougal', 'mackey', 'adrian']


['we', 'thank', 'dougal', 'mackey', 'adrian']